#Documentation

Please mount your drive (in the first block)

### How to test a trained model on multiple images

1. Go to "Test on multiple images" section.
2. Fill in the path to the test directory (TEST_PATH variable).
  * The script expects the test directory to be in this structue:
    * test_dir
      * test_set_1
      * test_set_2
      * test_set_3
3. Fill in the path for both the generator of Monet dataset and Photos dataset (GENERATOR_PATH_MONET, GENERATOR_PATH_PHOTOS).
4. Fill in (if needed. Right now they are just as the test we were given) the names of the test sets (the test sets refers to the directories in the test directory (test_set_1, test_set_2 and test_set_3 in the example above).
5. Run the block of the variables you filled and the block after that (which contains the functions)
6. Run the block that calls the "test" method. Pass as arguments a test set for your choice, and a boolean value indicating if you are running Monet dataset or Photos dataset (True for Monet, False for Photos).
  * When running Monet dataset, you need to run the block which contains the function "build_generator" (first block under "Declare Models" section).

### How to test a traind model on a single image

1. Go to "Test on single image" section.
2. Fill in the path to the image and the mask.
3. Fill in the path for both the generator of Monet dataset and Photos dataset (GENERATOR_PATH_MONET, GENERATOR_PATH_PHOTOS).
4. Run all the blocks in the section.
  * Pass as arguments a test set for your choice, and a boolean value indicating if you are running Monet dataset or Photos dataset (True for Monet, False for Photos).
  * When running Monet dataset, you need to run the block which contains the function "build_generator" (first block under "Declare Models" section).

# Preperation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import os
import imageio
from glob import glob
import matplotlib.pyplot as plt

from keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, BatchNormalization, Dropout, Activation, Flatten, Dense, MaxPooling2D, Permute, Reshape, Conv2DTranspose, Lambda
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, Sequential, load_model
import keras.layers as layers
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from keras import backend as K
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img, save_img



In [ ]:
DATASET_MONET_PATH = '/content/drive/MyDrive/deep learning data/data/monet'
DATASET_PHOTOS_PATH = '/content/drive/MyDrive/deep learning data/data/photos'
MASK_PATH = '/content/drive/MyDrive/deep learning data/masks'

# Declare Models

In [ ]:
IMG_SHAPE = (256, 256, 3)
PATCH_SIZE = 3
MASK_SIZE = 128

In [ ]:
IMG_SHAPE = (256, 256, 3)

def build_generator():
  encoder_input = layers.Input(shape=IMG_SHAPE)
  c1 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='conv1')(encoder_input)
  c1 = layers.Dropout(0.1)(c1)
  c1 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = layers.MaxPooling2D((2, 2))(c1)

  c2 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='conv2')(p1)
  c2 = layers.Dropout(0.1)(c2)
  c2 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = layers.MaxPooling2D((2, 2))(c2)

  c3 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='conv3')(p2)
  c3 = layers.Dropout(0.2)(c3)
  c3 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = layers.MaxPooling2D((2, 2))(c3)

  c4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = layers.Dropout(0.2)(c4)
  c4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
  c5 = layers.Dropout(0.3)(c5)
  c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

  # Expansive path 
  u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = layers.concatenate([u6, c4])
  c6 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = layers.Dropout(0.2)(c6)
  c6 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

  u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = layers.concatenate([u7, c3])
  c7 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = layers.Dropout(0.2)(c7)
  c7 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

  u8 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = layers.concatenate([u8, c2])
  c8 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = layers.Dropout(0.1)(c8)
  c8 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

  u9 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = layers.concatenate([u9, c1], axis=3)
  c9 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
  c9 = layers.Dropout(0.1)(c9)
  c9 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

  outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c9)

  return Model(inputs=[encoder_input], outputs=[outputs])

In [ ]:
def build_discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=IMG_SHAPE, name='input_image')

  result = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer, use_bias=False)(inp)
  result = LeakyReLU(alpha=0.2)(result)
  result = MaxPooling2D((2, 2), padding="same")(result)
  result = BatchNormalization(momentum=0.8)(result)

  result = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer, use_bias=False)(result)
  result = LeakyReLU(alpha=0.2)(result)
  result = MaxPooling2D((2, 2), padding="same")(result)
  result = BatchNormalization(momentum=0.8)(result)

  result = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer, use_bias=False)(result)
  result = LeakyReLU(alpha=0.2)(result)
  result = MaxPooling2D((2, 2), padding="same")(result)
  result = BatchNormalization(momentum=0.8)(result)
  
  result = tf.keras.layers.ZeroPadding2D()(result) 
  result = tf.keras.layers.Conv2D(512, (4,4), strides=1, kernel_initializer=initializer, use_bias=False)(result) 

  result = tf.keras.layers.BatchNormalization()(result)
  result = tf.keras.layers.LeakyReLU()(result)
                                
  patch_out = Activation('sigmoid')(result)

  return tf.keras.Model(inputs=inp, outputs=result)

In [ ]:
def build_combined_generator_photo(generator, discriminator):
  discriminator.trainable = False
  combined_input = Input(shape=IMG_SHAPE)
  generated_img = generator(combined_input)
  is_real = discriminator(generated_img)
  generated_img = Lambda(K.identity, name='generator_output')(generated_img)
  is_real = Lambda(K.identity, name='discriminator_output')(is_real)
  return Model(combined_input, [generated_img, is_real])

In [ ]:
def build_combined_generator_monet(generator, discriminator):
  discriminator.trainable = False
  combined_input = Input(shape=IMG_SHAPE)
  generated_output = generator(combined_input)
  is_real = discriminator(generated_output)
  generated_img = Lambda(K.identity, name='generator_output')(generated_output)
  generated_img2 = Lambda(K.identity, name='generator_output2')(generated_output)
  is_real = Lambda(K.identity, name='discriminator_output')(is_real)
  return Model(combined_input, [generated_img, generated_img2, is_real])

# Load Dataset

In [ ]:
BATCH_SIZE = 16
datagen_photos = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_imgs_photos = datagen_photos.flow_from_directory(
    DATASET_PHOTOS_PATH,
    class_mode=None,
    batch_size=BATCH_SIZE,
    subset='training',
    shuffle=False
)
val_imgs_photos = datagen_photos.flow_from_directory(
    DATASET_PHOTOS_PATH,
    class_mode=None,
    batch_size=BATCH_SIZE,
    subset='validation',
    shuffle=False
)


datagen_monet = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_imgs_monet = datagen_monet.flow_from_directory(
    DATASET_MONET_PATH,
    class_mode=None,
    batch_size=BATCH_SIZE,
    subset='training',
    shuffle=False
)

val_imgs_monet = datagen_monet.flow_from_directory(
    DATASET_MONET_PATH,
    class_mode=None,
    batch_size=BATCH_SIZE,
    subset='validation',
    shuffle=False
)

datagen_mask = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
masks = datagen_mask.flow_from_directory(
    MASK_PATH,
    class_mode=None,
    batch_size=BATCH_SIZE,
    subset='training',
    shuffle=True)

Found 830 images belonging to 1 classes.
Found 207 images belonging to 1 classes.
Found 384 images belonging to 1 classes.
Found 96 images belonging to 1 classes.
Found 60 images belonging to 1 classes.


# Create Masks

In [ ]:
def create_mask_images(images):
  mask_images = np.empty_like(images)
  for i, image in enumerate(images):
    masked_img = image.copy()
    x = np.random.randint(0, len(masks))
    y = np.random.randint(0, len(masks[x]))
    masked_img[masks[x][y] > 0.1] = 0
    mask_images[i] = masked_img
  return mask_images

#Style Loss

In [ ]:
STYLE_WEIGHT = 0.1

In [ ]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram


def layer_style_loss(style, generator_output):
    S = gram_matrix(style)
    C = gram_matrix(generator_output)
    channels = IMG_SHAPE[2]
    size = IMG_SHAPE[0] * IMG_SHAPE[1]
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))


def style_loss(model, style_layer_names):
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
    feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)

    def compute_loss(generator_output, style_image):
        input_tensor = tf.concat(
            [generator_output, style_image], axis=0
        )
        features = feature_extractor(input_tensor)

        loss = tf.zeros(shape=())

        for layer_name in style_layer_names:
            layer_features = features[layer_name]
            style_reference_features = layer_features[1, :, :, :]
            combination_features = layer_features[0, :, :, :]
            sl = layer_style_loss(style_reference_features, combination_features)
            loss += (1 / len(style_layer_names)) * sl
        return loss
    return compute_loss

#Compile Models

In [ ]:
def create_model_photo():
  generator = build_generator()
  generator.compile(loss='mse',
                    optimizer='adam')

  discriminator_copy = build_discriminator()
  discriminator_copy.compile(loss='binary_crossentropy',
                            optimizer='adam')

  discriminator = discriminator_copy

  combined_generator = build_combined_generator_photo(generator, discriminator_copy)
  combined_generator.compile(loss=['mse', 'binary_crossentropy'],
                            optimizer='adam',
                            loss_weights=[0.999, 0.001])
  
  return generator, discriminator, combined_generator

In [ ]:
def create_model_monet():
  generator = build_generator()
  style_layer_names = [
    'conv1',
    'conv2',
    'conv3'
  ]
  generator.compile(optimizer='adam',
                    loss=['mse', style_loss(generator, style_layer_names)],
                    loss_weights=[1 - STYLE_WEIGHT, STYLE_WEIGHT])

  discriminator_copy = build_discriminator()
  discriminator_copy.compile(loss='binary_crossentropy',
                            optimizer='adam')

  discriminator = discriminator_copy

  combined_generator = build_combined_generator_monet(generator, discriminator_copy)
  combined_generator.compile(loss=['mse', style_loss(generator, style_layer_names), 'binary_crossentropy'],
                            optimizer='adam',
                            loss_weights=[0.998, 0.001, 0.001])
  return generator, discriminator, combined_generator

# Train

In [ ]:
def create_models_and_datasets(is_monet):
  if is_monet:
    generator, discriminator, combined_generator = create_model_monet()
    train_imgs = train_imgs_monet
    val_imgs = val_imgs_monet
  else:
    generator, discriminator, combined_generator = create_model_photo()
    train_imgs = train_imgs_photos
    val_imgs = val_imgs_photos
  return  generator, discriminator, combined_generator, train_imgs, val_imgs

In [ ]:
def train(is_monet, generator, discriminator, combined_generator, train_imgs, val_imgs):
  epochs = 1 # todo
  train_loss = []
  val_loss = []
  for i in range(epochs + 1):
    print('epoch:' + str(i))
    idx = random.sample(range(len(train_imgs)), len(train_imgs))
    count = 0
    batch_train_loss = 0
    batch_val_loss = 0

    for j in range(len(train_imgs)):
      real = np.empty((train_imgs[j].shape[0], PATCH_SIZE, PATCH_SIZE, 1))
      real.fill(0.8)
      fake = np.empty((train_imgs[j].shape[0], PATCH_SIZE, PATCH_SIZE, 1))
      fake.fill(0.2)
      masked_imgs = create_mask_images(train_imgs[j])
      gen_missing = generator.predict(masked_imgs)
      d_loss_real = discriminator.train_on_batch(train_imgs[j], real)
      d_loss_fake = discriminator.train_on_batch(gen_missing, fake)
      if is_monet:
        g_loss = combined_generator.train_on_batch(masked_imgs, [train_imgs[j], train_imgs[j], real])
      else:
        g_loss = combined_generator.train_on_batch(masked_imgs, [train_imgs[j], real])
      batch_train_loss += g_loss[1]
      count += 1

    for j in range(len(val_imgs)):
      real = np.empty((val_imgs[j].shape[0], PATCH_SIZE, PATCH_SIZE, 1))
      real.fill(0.8)
      fake = np.empty((val_imgs[j].shape[0], PATCH_SIZE, PATCH_SIZE, 1))
      fake.fill(0.2)
      masked_imgs = create_mask_images(val_imgs[j])
      gen_missing = generator.predict(masked_imgs)
      if is_monet:
        g_loss = combined_generator.test_on_batch(masked_imgs, [val_imgs[j], val_imgs[j], real])
      else:
        g_loss = combined_generator.test_on_batch(masked_imgs, [val_imgs[j], real])
      batch_val_loss += g_loss[1]

    print(f'train loss: {batch_train_loss / len(train_imgs)}, val loss: {batch_val_loss / len(val_imgs)}')
    train_loss.append(batch_train_loss / len(train_imgs))
    val_loss.append(batch_val_loss / len(val_imgs))

In [ ]:
# is_monet = False
is_monet = True

generator, discriminator, combined_generator, train_imgs, val_imgs = create_models_and_datasets(is_monet=is_monet)

train(is_monet, generator, discriminator, combined_generator, train_imgs, val_imgs)

epoch:0
train loss: 0.018658598187771117, val loss: 0.009005992615129799
saving
epoch:1
train loss: 0.008408413861285557, val loss: 0.007097969154446301
epoch:2
train loss: 0.006260364778096449, val loss: 0.005922251822829077
epoch:3
train loss: 0.0066670823467905975, val loss: 0.005222031986870041
epoch:4
train loss: 0.005498258908284532, val loss: 0.0043410395340867
epoch:5
train loss: 0.004680037852350241, val loss: 0.0039251019638455046
saving
epoch:6
train loss: 0.004395856602059212, val loss: 0.003929231098895384
epoch:7
train loss: 0.0041147409342970195, val loss: 0.008959797687235881
epoch:8
train loss: 0.004400331296809864, val loss: 0.0039058241395237433
epoch:9
train loss: 0.004106013822655024, val loss: 0.0038037268148565836
epoch:10
train loss: 0.0038585256704457356, val loss: 0.0034055594428950412
saving
epoch:11
train loss: 0.0036464795292968947, val loss: 0.004353094851301814
epoch:12
train loss: 0.0035869089664298704, val loss: 0.0033186185544102705
epoch:13
train loss

# Test

### Test on multiple images

In [ ]:
from PIL import Image
from keras.preprocessing.image import img_to_array, load_img, save_img
from keras.models import load_model
import os
import numpy as np

In [ ]:
TEST_PATH = '/content/drive/MyDrive/deep learning data/test' # todo enter test path

MONET_BLOCKS = 'monet_blocks'
MONET_CENTRAL_BLOCK = 'monet_central_block'
PHOTOS_BLOCKS = 'photos_blocks'
PHOTOS_CENTRAL_BLOCK = 'photos_central_block'
PHOTOS_REGION = 'photos_region'

GENERATOR_PATH_MONET = '/content/drive/MyDrive/deep learning/models_final/generator_final_model_2_monet.h5' # todo enter path to generator
GENERATOR_PATH_PHOTOS = '/content/drive/MyDrive/deep learning data/models/generator_final_model_1_photos.h5' # todo enter path to generator

RESULTS_PATH = '' # todo enter path results directory

In [ ]:
def save_results(images, results_path, mask_type):
  for i, img in enumerate(images):
    save_img(f'{results_path}/{mask_type}_{i}.jpg', img)

def put_generated_parts_on_original_images(originals, generated, masks):
  results = np.empty_like(originals)
  for i in range(len(originals)):
    original_copy = originals[i].copy()
    original_copy[masks[i] > 0.95] = generated[i][masks[i] > 0.95]
    results[i] = original_copy
  return results

def turn_mask_to_black(images, masks):
  results = np.empty_like(images)
  for i, img in enumerate(images):
    img_copy = img.copy()
    img_copy[masks[i] > 0.95] = 0
    results[i] = img_copy
  return results

def load_test_set_and_masks(test_path):
  test_set = {}
  masks = {}
  for filename in os.listdir(test_path):
    if not filename.endswith("jpg"): 
      continue
    img = load_img(f'{test_path}/{filename}')
    img = img.resize((256, 256))
    if "mask" in filename:
      masks[filename] = img_to_array(img)
    else:
      test_set[filename] = img_to_array(img)
  test_names = [k for k in test_set.keys()]
  test_names.sort()
  test_set = [test_set[x] for x in test_names]
  masks_names = [k for k in masks.keys()]
  masks_names.sort()
  masks = [masks[x] for x in masks_names]
  return np.array(test_set) / 255, np.array(masks) / 255


def test(mask_type, is_monet):
  if is_monet:
    # generator = load_model(GENERATOR_PATH_MONET)

    generator = build_generator()
    generator.load_weights(GENERATOR_PATH_MONET)
  else:
    generator = load_model(GENERATOR_PATH_PHOTOS)

  test_set, masks = load_test_set_and_masks(f'{TEST_PATH}/{mask_type}')
  test_set = turn_mask_to_black(test_set, masks)
  results = generator(test_set)
  results = put_generated_parts_on_original_images(test_set, results, masks)
  save_results(results, RESULTS_PATH, mask_type)

In [ ]:
# test(MONET_CENTRAL_BLOCK, True)
test(PHOTOS_CENTRAL_BLOCK, False)

OSError: ignored

### Test on single image

In [ ]:
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array, load_img, save_img
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
GENERATOR_PATH_MONET = '/content/drive/MyDrive/deep learning/models_final/generator_final_model_2_monet.h5' # todo enter path to Monet generator
GENERATOR_PATH_PHOTOS = '/content/drive/MyDrive/deep learning data/models/generator_final_model_1_photos.h5' # todo enter path to Photos generator
IMAGE_PATH = '/content/drive/MyDrive/photo/photo_jpg/fff5c33050.jpg' # enter path to test image
MASK_PATH = '/content/drive/MyDrive/deep learning data/masks/masks/mask3.jpg' # enter path to test mask

In [ ]:
def load_image_and_mask(image_path, mask_path):
  img = load_img(image_path)
  img = img.resize((256, 256))
  img = img_to_array(img) / 255
  mask = load_img(mask_path)
  mask = mask.resize((256, 256))
  mask = img_to_array(mask) / 255
  return img, mask

def turn_mask_to_black(image, mask):
  img_copy = image.copy()
  img_copy[mask > 0.95] = 0
  return img_copy

def put_generated_parts_on_original(original, generated, mask):
  original_copy = original.copy()
  original_copy[mask > 0.95] = generated[0][mask > 0.95]
  return original_copy

def show_result(result):
  plt.imshow(result)
  plt.show()

In [ ]:
def test_single_image(image_path, mask_path, is_monet):
  if is_monet:
    generator = build_generator()
    generator.load_weights(GENERATOR_PATH_MONET)
  else:
    generator = load_model(GENERATOR_PATH_PHOTOS)

  img, mask = load_image_and_mask(image_path, mask_path)
  img = turn_mask_to_black(img, mask)
  result = generator(np.array([img]))
  result = put_generated_parts_on_original(img, result, mask)
  show_result(result)

In [ ]:
test_single_image(IMAGE_PATH, MASK_PATH, False)
# test_single_image(IMAGE_PATH, MASK_PATH, True)

OSError: ignored